In [27]:
import google.cloud.vision as gcv
from google.oauth2 import service_account
from google.protobuf.json_format import MessageToDict
import json
import langcodes

In [3]:
web_detection_params = gcv.types.WebDetectionParams(include_geo_results=True)
image_context = gcv.types.ImageContext(web_detection_params=web_detection_params)

In [4]:
# credentials must be loaded as below, otherwise there will be an error
credentials = service_account.Credentials.from_service_account_file('credentials/ArnottsAU-7991416de13b.json')

In [5]:
client = gcv.ImageAnnotatorClient(credentials=credentials)

In [7]:
# read image file as binary
img = gcv.types.Image(content=open('pictures/picture_582887468579563.jpg', 'rb').read())

In [8]:
f = open('pictures/picture_582887468579563.jpg', 'rb').read()

r = MessageToDict(client.annotate_image({'image': 
                       {'content': f}, 'image_context': image_context}), 
                  preserving_proto_field_name = True)

In [9]:
face_feats = 'joy sorrow anger surprise under_exposed blurred headwear'.split()

In [16]:
def count(what):
    _ = r.get(f'{what}_annotations', None)
    print(f'found {len(_) if _ else 0} {what}(s)')

### Face

In [17]:
count('face')

found 1 face(s)


In [18]:
for e in face_feats:
    print(f'{e}: {r["face_annotations"][0].get(e + "_likelihood", None)}')

joy: VERY_UNLIKELY
sorrow: VERY_UNLIKELY
anger: VERY_UNLIKELY
surprise: VERY_UNLIKELY
under_exposed: VERY_UNLIKELY
blurred: VERY_UNLIKELY
headwear: VERY_UNLIKELY


### Logos

In [19]:
count('logo')

found 0 logo(s)


In [189]:
r['logo_annotations'][0]['description']

'Tim Tam'

### Labels

In [20]:
count('label')

found 10 label(s)


In [21]:
for l in r['label_annotations']:
    print(f'{l["description"]}, score: {l["score"]}')

blue, score: 0.9681232571601868
pink, score: 0.9571918845176697
red, score: 0.9551005959510803
shoulder, score: 0.8600255250930786
fun, score: 0.8313401937484741
girl, score: 0.7911158204078674
standing, score: 0.7800707221031189
arm, score: 0.7518427968025208
human body, score: 0.7411763668060303
mouth, score: 0.6754465103149414


### Text in Picture

In [22]:
count('text')

found 3 text(s)


In [29]:
for i, t in enumerate(r['text_annotations'], 1):
    print(f'#{i} -- language: {langcodes.Language.make(language=t["locale"]).language_name() if "locale" in t  else "?"}, text: {t["description"]}')

#1 -- language: Cebuano, text: non SA

#2 -- language: ?, text: non
#3 -- language: ?, text: SA


### Restricted Themes

In [24]:
r['safe_search_annotation']

{'adult': 'UNLIKELY',
 'spoof': 'VERY_UNLIKELY',
 'medical': 'VERY_UNLIKELY',
 'violence': 'VERY_UNLIKELY',
 'racy': 'POSSIBLE'}

### Colors
higher "scores" means higher confidence that the color in question is prominent in the central focus of the image

In [25]:
for c in r['image_properties_annotation']['dominant_colors']['colors']:
    print(c['color'], c['pixel_fraction'], c['score'])

{'red': 164.0, 'green': 200.0, 'blue': 202.0} 0.12246666848659515 0.1612282544374466
{'red': 106.0, 'green': 121.0, 'blue': 124.0} 0.13046666979789734 0.10734815895557404
{'red': 197.0, 'green': 68.0, 'blue': 90.0} 0.041866667568683624 0.06679261475801468
{'red': 15.0, 'green': 16.0, 'blue': 33.0} 0.030266666784882545 0.04429831728339195
{'red': 14.0, 'green': 24.0, 'blue': 159.0} 0.0019333333475515246 0.011289363726973534
{'red': 50.0, 'green': 66.0, 'blue': 190.0} 0.0007999999797903001 0.004694235511124134
{'red': 27.0, 'green': 35.0, 'blue': 130.0} 0.0007999999797903001 0.003713528160005808
{'red': 136.0, 'green': 167.0, 'blue': 169.0} 0.12460000067949295 0.11548937112092972
{'red': 34.0, 'green': 50.0, 'blue': 201.0} 0.00013333333481568843 0.0008799362694844604
{'red': 130.0, 'green': 149.0, 'blue': 242.0} 0.00019999999494757503 0.0007876573945395648


In [26]:
r['full_text_annotation']['pages'][0]['property']['detected_languages']

[{'language_code': 'ceb', 'confidence': 1.0}]

### Full Text Annotations

In [199]:
print('pages: ', len(r['full_text_annotation']['pages']))

pages:  1


In [200]:
for p in range(len(r['full_text_annotation']['pages'])):
    print(r['full_text_annotation']['pages'][p]['property']['detected_languages'])

[{'language_code': 'en', 'confidence': 0.7699999809265137}]


In [201]:
r['full_text_annotation']['text']

'TimTam\nSuper Scrummy\nChocolate Tim Tam\nMilkshake\nREARNOTTS\nTimTam\nORIGINAL\n'

### Web Detection
This one gives us 
* web_entities
* visually_similar_images
* best_guess_labels
Note: there's an overall relevancy score for the entity, not normalized and not comparable across different image queries.

In [202]:
web_ents = len(r['web_detection']['web_entities'])
print(f'web entities found: {web_ents}')

web entities found: 10


In [203]:
for e in r['web_detection']['web_entities']:
    print(f'entity: {e["description"] if "description" in e else "?"}, score: {e["score"]}')

entity: Sundae, score: 1.578178882598877
entity: Tim Tam, score: 0.8896999955177307
entity: Liqueur, score: 0.8364270329475403
entity: ?, score: 0.6556000113487244
entity: Arnott's Biscuits, score: 0.6057000160217285
entity: Chocolate, score: 0.5485801696777344
entity: Flavor by Bob Holmes, Jonathan Yen (narrator) (9781515966647), score: 0.5430999994277954
entity: Biscuit, score: 0.5418000221252441
entity: Spain, score: 0.5078999996185303
entity: United Arab Emirates, score: 0.5071499943733215


In [204]:
sim_imgs = len(r['web_detection']['visually_similar_images'])
print(f'visually_similar_images found: {sim_imgs}')

visually_similar_images found: 10


In [205]:
r['web_detection']['visually_similar_images']

[{'url': 'https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=1916596088657356'},
 {'url': 'https://s314.siliconimg.com/kb/content_images/2017/12/13/1496558/1513170699_709.jpg'},
 {'url': 'http://www.uhainiu.com/content/images/thumbs/000/0000678_timtam-187g.jpeg'},
 {'url': 'https://cbu01.alicdn.com/img/ibank/2017/452/396/4547693254_937679173.jpg'},
 {'url': 'http://www.totallytarget.com/wp-content/uploads/2016/02/tim-tam-1.jpg'},
 {'url': 'https://wx1.sinaimg.cn/orj360/006YBhA8gy1fm1dmmcd4aj31he0u0aep.jpg'},
 {'url': 'https://www.campbellsoupcompany.com/wp-content/uploads/sites/31/2013/11/Tim-Tam-Chocolicious.jpg'},
 {'url': 'https://s1.bukalapak.com/img/139947173/large/Biskuit_Tim_Tam_Chocolate_100g_x_3_pcs.jpg'},
 {'url': 'https://media.apnarm.net.au/media/images/2014/02/16/tim_tam_50_years-lre5y1mnzhl8vk6nnh2_fct621x468_ct677x380.JPG'},
 {'url': 'https://www.c-store.com.au/wp-content/uploads/2015/07/Tim-Tam.jpg'}]

In [206]:
r['web_detection']['best_guess_labels']

[{'label': 'tim tam', 'language_code': 'en'}]

### Localized Objects

In [207]:
for o in r['localized_object_annotations']:
    print(f'{o["name"]}, score: {o["score"]}')

Drink, score: 0.5362362265586853
